# Track log evaluation

## Data
* [lag_data](https://drive.google.com/open?id=1Hc5bGXMLV3LqfnAwWKMPylfHothAHtPc)
* [lag_data1](https://drive.google.com/open?id=1ok6RiGLbfNGN-GYA3ejpwKE1TI7LSotE)

## Classification vs accumulation

![results](./image/Classification_results_size_ver2.png)

```
wo classification accuracy: 86.6279069767
w classification accuracy: 93.5483870968
wo classification accuracy: 447.0 / 516.0
w classification accuracy: 493.0 / 527.0
```

## Classification vs accumulated frame

![results](./image/results2.png)

```
results 1: 303.0 / 611.0 
results 2: 163.0 / 270.0 
results 3: 101.0 / 188.0 
results 4: 92.0 / 141.0 
results 5: 88.0 / 109.0 
results 6: 73.0 / 84.0 
results 7: 59.0 / 65.0 
results 8: 36.0 / 41.0 
```

In [1]:
import os, sys

import re

import matplotlib.pyplot as plt

import numpy as np


In [2]:
def track_log_parsing(file_path):
    
    track = {}
    track['id'] = []
    track['status'] = []
    track['skippedframe'] = []
    track['updatedframe'] = []
    track['accumulatedframe'] = []
    track['scopeframe'] = []
    track['x'] = []
    track['y'] = []
    track['z'] = []
    track['class'] = []
    track['pointnum'] = [] 
    track['timestamp'] = []
    
    f = open(file_path, 'r')
    lines = f.readlines()[1:]
    
    for line in lines:
        data = line.split(', ')
        
        for d in data:
            track[d.split(': ')[0]].append( float(d.split(': ')[1]) )
        
    f.close()
    
    return track

In [9]:
""" Classification vs time """

""" Path """

track_log_path_list = []

tlp = os.path.abspath('./log_data7/wo/log_track_0.txt')
track_log_path_list.append(tlp)

tlp = os.path.abspath('./log_data7/w/log_track_0.txt')
track_log_path_list.append(tlp)

""" Data parsing """

tracks = []

for tlp in track_log_path_list:
    
    track = track_log_parsing(tlp)
    tracks.append(track)
    
    """ Classification visualization """
wo_time = np.array(tracks[0]['timestamp']) - tracks[0]['timestamp'][0]
w_time = np.array(tracks[1]['timestamp']) - tracks[1]['timestamp'][0]

wo_acc_cls = np.array( tracks[0]['class'] )
w_acc_cls = np.array( tracks[1]['class'] )
wo_acc_cls[wo_acc_cls == 2] = 1
w_acc_cls[w_acc_cls == 2] = 1

%matplotlib qt
plt.close("all")
fig = plt.figure()

ax = fig.add_subplot(111)
ax.axvspan(0, 42, ymin=0.4, ymax=0.7, alpha=0.2, color='green', label="ground truth")

ax.fill_between(wo_time, wo_acc_cls, step="pre", alpha=1.0)
ax.plot(wo_time, wo_acc_cls, drawstyle="steps", label = 'w/o accumulation', marker = 'o', markersize = '3', linewidth = 1)

ax.fill_between(w_time, w_acc_cls + 0.1, step="pre", alpha=0.5)
ax.plot(w_time, w_acc_cls + 0.1, drawstyle="steps", color='red',label = 'w/ accumulation', marker = 's', markersize = '3', linewidth = 1)

ax.set_yticks([0, 0.5 , 1.0, 2.0])
ax.grid()
ax.set_title("Classification results", weight='bold')
ax.set_xlabel('time[sec]', weight='bold')
ax.set_xlim(0,30)
ax.set_ylabel('Class', fontsize=12, weight='bold')
ax.set_yticklabels( ['', 'unknown', 'car'], rotation=90)

plt.legend()
plt.show()

print "wo classification accuracy: {}".format( float ( np.sum(wo_acc_cls == 1) ) / float ( wo_acc_cls.shape[0]) * 100.0) 
print "w classification accuracy: {}".format( float ( np.sum(w_acc_cls == 1) ) / float ( w_acc_cls.shape[0]) * 100.0) 

print "wo classification accuracy: {} / {}".format( float ( np.sum(wo_acc_cls == 1) ) , float ( wo_acc_cls.shape[0]) )
print "w classification accuracy: {} / {}".format( float ( np.sum(w_acc_cls == 1) ) , float ( w_acc_cls.shape[0]) )


wo classification accuracy: 86.6279069767
w classification accuracy: 93.5483870968
wo classification accuracy: 447.0 / 516.0
w classification accuracy: 493.0 / 527.0


In [4]:
""" Accumulation vs accuracy """

""" Path """

track_log_path_list = []

tlp = os.path.abspath('./log_data2' )
track_log_path_list.append(tlp)


""" Data parsing """

tracks = []

for tlp in track_log_path_list:
        
    ## read all txt files
    for root, dirs, files in os.walk(tlp):
        rootpath = os.path.join(os.path.abspath(tlp), root)

        for file in files:
            if file.split('.')[-1] == 'txt':

                filepath = os.path.join(rootpath, file)
                track = track_log_parsing(filepath)
                
                tracks.append(track)
                
""" Data analysis """
t_num = [ 0. for _ in range(9)]
f_num = [ 0. for _ in range(9)]


## wo registration
for t in tracks:
    
    for i, c in enumerate( t['class'] ):
        
        if t['pointnum'][i]  < 30: continue
        
        if int( c ) == 1:
            if len(t_num) > int(t['accumulatedframe'][i]): 
                t_num[  int( t['accumulatedframe'][i] ) -1 ] +=1.
        else:
            if len(f_num) > int(t['accumulatedframe'][i]): 
                f_num[  int( t['accumulatedframe'][i] ) -1 ] +=1.
      
results = []
for i in range( len(t_num)):
    
    if float(t_num[i] + f_num[i]) == 0 : continue
        
    results.append( float(t_num[i]) / float(t_num[i] + f_num[i]) )
    print "results {}: {} / {} ".format(i+1, t_num[i] , t_num[i] +f_num[i] )

%matplotlib qt
plt.close("all")
fig = plt.figure()

ax = fig.add_subplot(111)

index = range(1, len(results) + 1)
ax.bar(index, results, align='center')
ax.grid()
ax.set_title("'Accuracy' vs '# of accumulated frames'", weight='bold')

# ax.set_xlabel('time[sec]', weight='bold')
# ax.set_xlim(0,8)
# ax.set_ylabel('Class', fontsize=12, weight='bold')
# ax.set_yticklabels( ['', 'unknown', 'car', 'truck'], rotation=90)

plt.ylabel('Accuracy', weight='bold')
plt.xlabel('# of accumulated frames', weight='bold')
# plt.xlim( 0,42)
# plt.ylim( 0, np.max(y_val) * 1.1 )

plt.legend()
plt.show()

results 1: 303.0 / 611.0 
results 2: 163.0 / 270.0 
results 3: 101.0 / 188.0 
results 4: 92.0 / 141.0 
results 5: 88.0 / 109.0 
results 6: 73.0 / 84.0 
results 7: 59.0 / 65.0 
results 8: 36.0 / 41.0 


In [12]:
results

[0.27671232876712326,
 0.6037037037037037,
 0.5372340425531915,
 0.6524822695035462,
 0.8073394495412844,
 0.8690476190476191,
 0.9076923076923077,
 0.8780487804878049,
 0.8333333333333334]